# Tutorial 5: Control Design
Welcome to Tutorial 5, where we will be looking at controller design. The first part of the tutorial will be focusing on PID control, you will implement and tune a PID controller for a one DOF system. The second part of the tutorial will be looking at LQR control for an inverted pendulum.

# Part A: PID Control

## Import Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

## Outline of Part A
The first part of the tutorial looks at PID control. Below are some objectives to be achieved by the end of the tutorial.

**Objectives:**
* Implement the PID control equations.
* Tune the PID controller gains to achieve a suitable response.
* Gain an intuitive understanding of how changing control gains affects the system response.

## PID Control

A proportional–integral–derivative controller (PID controller) is a type of feedback or closed-loop controller that is widely used in industrial control systems. The PID controller calculates an error $e(t)$ at each time step $h$. This error value is the difference between a reference or target $r(t)$ and an actual output or controlled variable $y(t)$. The controller works by reducing the error by adjusting the control variable $u(t)$, based on the weighted sum of the proportional, integral and derivative terms with their respective gains. 

An everday example of feedback control is cruise control, where the driver sets a target speed and the controller has to work to reduce the error between its current  and target speed. The cruise controller may have other requirements such as having minimal delay and not overshooting the target speed. 

<img src="images/PID.svg" style="width: 700px;">

<br/>
<img src="images/continuous_PID.png" style="width: 300px;">

## PID Gain Tuning
There are a number of methods for tuning a PID controller to get a desired response. Below is a summary of how increasing each of the control gains affects the response:


| Parameter | Rise time    | Overshoot  | Settling time | Steady-state error | Stability |
| :---------|:------------:| ----------:|:-------------:|:------------------:| :--------:|
| $K_p$     | Decrease     | Increase   | Small change  | Decrease           | Degrade   |
| $K_i$     | Decrease     | Increase   | Increase      | Eliminate          | Degrade   |
| $K_d$     | Minor change | Decrease   | Decrease      | No effect          | Improve if $K_d$ small |

In this tutorial, we will be implementing and tuning the PID controller for a one DOF system. This could for example represent a particle moving vertically in one dimension under an acceleration.

### Tasks:
* Implement the PID control loop equations
* Tune the gains of the PID loop

### Implement PID Controller

In [ ]:
class OneDofSystem(object):
    def __init__(self, x, v, biasAcceleration=0):
        self.x = x
        self.v = v
        self.biasAcceleration = biasAcceleration
        
    def step(self, u, h):
        # Returns the updated position x and velocity v after one time step h
        a = self.biasAcceleration + u
        self.x = self.x + h*self.v + 1./2.*a*h**2 # EOM for position
        self.v = self.v + h*a # EOM for velocity
        
    def sense(self):
        # Returns the current position
        return self.x
    
def response(system, k_p, k_d, k_i):
    sys = system(0, 0)
        
    x_target = 1 # Set target position
    delta_x_target = 1 # Updating the target 
    
    h = 0.01 # dt
    x_old = sys.sense() # initial position of the system
    
    e_old = 0 # initial error
    e_i = 0 # initial intergral error
    
    x_list = []
    target_list = []
 
    for c in range(1000):       
        # obtain current position of system         
        x = sys.sense()
        
        # half-way update the target         
        if c==500:
            x_target += delta_x_target
        
        # Implement the discretized PID loop equations
        ### START CODE HERE ### (~ 4 lines of code)
        e = None   #Calculate the error in position 
        e_d = None # Calculate derivative of the error
        e_i = None # Calculate integral of the error  
        u = None   # Calculate the output of the PID loop
        ### END CODE HERE ###
        
        # apply control effort, u (acceleration)
        sys.step(u, h)
        
        x_old = x # store previous position 
        e_old = e # store previous error 
        
        # store position, target and time for plotting reasons         
        x_list.append(x)
        target_list.append(x_target)
        time = np.arange(1000)*h
    return time, np.array(x_list), np.array(target_list)

# Tune the PID equations
### START CODE HERE ### (3 lines of code)
k_p = None
k_d = None
k_i = None
### END CODE HERE ###

systemFun = lambda x, v : OneDofSystem(x,v, 0)
time, position, target = response(systemFun, k_p, k_d, k_i)
    
%matplotlib notebook        
    
plt.plot(time, position)
plt.plot(time,np.ones(time.shape)*target)
plt.xlabel("time (in s)")
plt.ylabel("position (in m)")

plt.show()   

## Questions

1. What happens with only a proportional gain and why? (P)
2. What is the effect of adding $K_d$ gain? (PD)
3. How do you create a faster response? (PD)
4. What happens when the $K_d$ is too high? (PD)
5. What is the effect of the integral gain $K_i$? (PID)
6. When would you use an integral gain? (PID)
7. What are two downsides of using the integral gain? (PID)

***


# Part B: LQR Control


## Outline of Part B

In the second part of the tutorial, we will be looking at the Linear-quadractic Regulator (LQR), another type of feedback controller. Here are the learning objectives for Part B.

**Objectives:**
* Write the inverted pendulum's dynamic equations in state-variable form.
* Implement a LQR controller.

## Inverted Pendulum or Cart Pole

<img src="images/inv_pend.png" style="width: 300px;">

The differential equations that describe the dynamics of the inverted pendulum/cart pole:
<br />
<br />
<center>$\ddot{x} = \frac{-mLsin(\theta)\dot{\theta}^2 + f + mgcos(\theta)sin(\theta)}{M + m - mcos(\theta)^2}$ </center>
<center>$\ddot{\theta} = \frac{-mLcos(\theta)sin(\theta)\dot{\theta}^2 + fcos(\theta) + (M+m)gsin(\theta)}{L(M + m - mcos(\theta)^2)}$ </center>

The linearized differential equations that describe the inverted pendulum/cart pole at $\theta = 0 $:
<br />
<br />
<center>$\ddot{x}$ $= \frac{f + m \theta g}{M}$</center>

<br />

<center>$\ddot{\theta}$ $= \frac{f + (M+m)g \theta}{LM}$</center>
<br />
Where: 

* $M$ - mass of the cart
* $m$ - mass of the pendulum/pole
* $L$ - length of the pendulum/pole
* $x$ - displacement of the cart
* $\theta$ - angle of the pendulum/pole
* $f$ - input force

In [ ]:
import numpy as np,cv2,math,time,matplotlib.pyplot as plt,sys

class Cart:
    def __init__(self,x,mass,world_size):
        self.x = x
        self.y = int(0.6*world_size) 
        self.mass = mass
        self.color = (0,255,0)

class Pendulum:
    def __init__(self,length,theta,ball_mass):
        self.length = length
        self.theta = theta
        self.ball_mass = ball_mass
        self.color = (0,0,255)

def display_stuff(world_size,cart,pendulum,count):
    # This function displays the pendulum and cart.
    length_for_display = pendulum.length * 100
    A = np.zeros((world_size,world_size,3),np.uint8)
    cv2.line(A,(0,int(0.6 * world_size)),(world_size,int(0.6 * world_size)),(255,255,255),2)
    cv2.rectangle(A,(int(cart.x) + 25,cart.y + 15),(int(cart.x) - 25,cart.y - 15),cart.color,-1)
    pendulum_x_endpoint = int(cart.x - (length_for_display) * math.sin(pendulum.theta))
    pendulum_y_endpoint = int(cart.y - (length_for_display) * math.cos(pendulum.theta))
    cv2.line(A,(int(cart.x),cart.y),(pendulum_x_endpoint,pendulum_y_endpoint),pendulum.color,4)
    cv2.circle(A,(pendulum_x_endpoint,pendulum_y_endpoint),6,(255,255,255),-1)
    cv2.imshow('Pendulum',A)
    cv2.waitKey(1)
    if count >=999:
        cv2.destroyAllWindows()


def apply_control_input(cart,pendulum,F,time_delta,x_tminus2,theta_dot,theta_tminus2,previous_time_delta,g):
    # Finding x and theta given the control inputs and the dynamics of the system
    theta_double_dot = (((cart.mass + pendulum.ball_mass) * g * math.sin(pendulum.theta)) + (F * math.cos(pendulum.theta)) - (pendulum.ball_mass * ((theta_dot)**2.0) * pendulum.length * math.sin(pendulum.theta) * math.cos(pendulum.theta))) / (pendulum.length * (cart.mass + (pendulum.ball_mass * (math.sin(pendulum.theta)**2.0))))
    x_double_dot = ((pendulum.ball_mass * g * math.sin(pendulum.theta) * math.cos(pendulum.theta)) - (pendulum.ball_mass * pendulum.length * math.sin(pendulum.theta) * (theta_dot**2)) + (F)) / (cart.mass + (pendulum.ball_mass * (math.sin(pendulum.theta)**2)))
    cart.x += ((time_delta**2) * x_double_dot) + (((cart.x - x_tminus2) * time_delta) / previous_time_delta)
    pendulum.theta += ((time_delta**2)*theta_double_dot) + (((pendulum.theta - theta_tminus2)*time_delta)/previous_time_delta)

def plot_graphs(times,theta,force,x,gain):
    
    # This function plots all the graphs
    plt.subplot(5, 1, 1)
    plt.plot(times,gain[:,0],'-b')
    plt.ylabel('GainX')
    plt.xlabel('Time')
    
    plt.subplot(5, 1, 2)
    plt.plot(times,gain[:,2],'-b')
    plt.ylabel('GainTheta')
    plt.xlabel('Time')
    
    plt.subplot(5, 1, 3)
    plt.plot(times,theta,'-b')
    plt.ylabel('Theta')
    plt.xlabel('Time')

    plt.subplot(5, 1, 4)
    plt.plot(times,force,'-b')
    plt.ylabel('Force')
    plt.xlabel('Time')

    plt.subplot(5, 1, 5)
    plt.plot(times,x,'-b')
    plt.ylabel('X')
    plt.xlabel('Time')

    plt.show()

In [ ]:
def main(control_technique):
    # Initializing mass values, g, world size, simulation time and variables required to terminate the simulation
    mass_of_ball = 1.0
    mass_of_cart = 5.0
    g = 9.81
    errors, force, theta, times, x, gain = [],[],[],[],[],[]
    world_size = 500

    # Initializing cart and pendulum objects
    cart = Cart(int(0.5 * world_size),mass_of_cart,world_size)
    pendulum = Pendulum(1,-1,mass_of_ball)

    # Initializing other variables needed for the simulation
    theta_dot = 0
    theta_tminus1 = theta_tminus2 = pendulum.theta
    x_tminus1 = x_tminus2 = cart.x

    delta_timestamp = 0.01
    previous_time_delta = delta_timestamp
    time_delta = delta_timestamp
    current_timestamp = 0

    count = 0
    # The simulation must run for the desired amount of time
    while count<1000:

        current_timestamp += current_timestamp + delta_timestamp

        theta_dot = (theta_tminus1 - theta_tminus2 ) / previous_time_delta
        x_dot = (x_tminus1 - x_tminus2) / previous_time_delta
        
        if control_technique == "lqr":
            f,k = find_lqr_control_input(cart,pendulum,theta_dot,x_dot,g)
        elif control_technique == "noforce":
            f = 0 
            k = np.zeros((1,4)) 
            
        # integration         
        apply_control_input(cart,pendulum,f,time_delta,x_tminus2,theta_dot,theta_tminus2,previous_time_delta,g)

        # For plotting the graphs
        force.append(f)
        gain.append(k.T)
        x.append(cart.x)
        times.append(current_timestamp)
        theta.append(pendulum.theta)

        # Update the variables and display stuff
        display_stuff(world_size,cart,pendulum,count)

        theta_tminus2 = theta_tminus1
        theta_tminus1 = pendulum.theta
        x_tminus2 = x_tminus1
        x_tminus1 = cart.x
        count += 1

    plot_graphs(times[:50],theta[:50],force[:50],x[:50],np.array(gain[:50]))

### Execute main function to simulate the system without any force applied on it

In [ ]:
%matplotlib notebook

main('noforce')

## LQR Control
An LQR controller can be used when a system's dynamics can be described by a set of **linear** differential equations and the cost is described by a **quadratic** function. 

The first step in creating our inverted pendulum LQR controller is to represent the dynamics equations of the system in **state-space form**:
<br />
<br />
<center>$\dot{x} = Ax +Bu$</center>

The second step is to define a **quadratic cost** function:
<br />
<center>$J = \int_0^{\infty} x^TQx + u^TRu$</center>

Where:
* **$\mathbf{u}$** - **control**
* **$\mathbf{x}$** - **state of the system**
* **$A$** - **system matrix**
* **$B$** - **input matrix**
* **$Q$** - **input matrix**
* **$R$** - **input matrix**
<br />
<br />

For the **Cart Pole Balancing** Problem:

* **control** - **$\mathbf{u} = f$** 
* **state of the system** - $\mathbf{x} = [x, \dot{x}, \theta, \dot{\theta}]^T$
* **dynamics of the system** are :
<br />

$$
\dot{\mathbf{x}}=
\left(\begin{array}{cc}
\dot{x} \\
\ddot{x} \\
\dot{\theta} \\
\ddot{\theta}
\end{array}\right) 
=
\left(\begin{array}{cc}
\dot{x}\\
\frac{f + m \theta g}{M} \\
\dot{\theta} \\
\frac{f + (M+m)g \theta}{LM}
\end{array}\right)
=
A
\left(\begin{array}{cc}
x \\
\dot{x} \\
\theta \\
\dot{\theta} \\
\end{array}\right) 
+
\left(\begin{array}{cc}
0 \\
\frac{1}{M} \\
0 \\
\frac{1}{LM}\\
\end{array}\right) 
f
$$ 

where in the RHS (right hand side) of the equation the **dynamics of the system** are written in **state-space form**.
<br />

### Task: 
* Find the 4x4 system matrix $A$ 

#### Answer:

$$
\dot{\mathbf{x}}=
\left(\begin{array}{cc}
\dot{x} \\
\ddot{x} \\
\dot{\theta} \\
\ddot{\theta}
\end{array}\right) 
=
\left(\begin{array}{cc}
\dot{x}\\
\frac{f + m \theta g}{M} \\
\dot{\theta} \\
\frac{f + (M+m)g \theta}{LM}
\end{array}\right)
=
\left(\begin{array}{cc}
? & ? & ? & ? \\
? & ? & ? & ? \\
? & ? & ? & ? \\
? & ? & ? & ? \\
\end{array}\right) 
\left(\begin{array}{cc}
x \\
\dot{x} \\
\theta \\
\dot{\theta} \\
\end{array}\right) 
+
\left(\begin{array}{cc}
0 \\
\frac{1}{M} \\
0 \\
\frac{1}{LM}\\
\end{array}\right) 
f
$$ 

and the requirements for the controller of the cart pole system are:
* We always want the pole upright - $\theta \approx 0$
* We always want the cart to be close to a reference point - $ x \approx x^*$
* We want to spend minimal energy (as small forces as possible) - $ f \approx 0$
<br />

Which translates to the following cost function: 

<br />

<center>$J = \int_0^{\infty} q_{\theta}||\theta||^2 + q_x||x^* - x||^2 + r||f||^2$</center>

<br />

Solving the optimal control (LQ) problem describe above, we obtain the control:

<br />

<center>$f = -K(x - x^*) $</center>

where K can be obtained by solving the Ricatti Differential Equation.

### Tasks: 
* Implement $A$ in the *find_lqr_control_input* function and run the simulation with the controller
* Try solving the problem with both the continuous and the discrete LQR (Ricatti) functions provided below.
* What do you observe?

In [ ]:
from scipy import signal


def lqr(A,B,Q,R):
    """Solve the continuous time lqr controller.
     
    dx/dt = A x + B u
     
    cost = integral x.T*Q*x + u.T*R*u
    """
    #ref Bertsekas, p.151
 
    #first, try to solve the ricatti equation
    X = np.matrix(scipy.linalg.solve_continuous_are(A, B, Q, R))
     
    #compute the LQR gain
    K = np.matrix(scipy.linalg.inv(R)*(B.T*X))
     
    eigVals, eigVecs = scipy.linalg.eig(A-B*K)
     
    return K, X, eigVals
 
def dlqr(A,B,Q,R):
    """Solve the discrete time lqr controller.
     
     
    x[k+1] = A x[k] + B u[k]
     
    cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k]
    """
    #ref Bertsekas, p.151
 
    #first, try to solve the ricatti equation
    X = np.matrix(scipy.linalg.solve_discrete_are(A, B, Q, R))
     
    #compute the LQR gain
    K = np.matrix(scipy.linalg.inv(B.T*X*B+R)*(B.T*X*A))
     
    eigVals, eigVecs = scipy.linalg.eig(A-B*K)
     
    return K, X, eigVals


def find_lqr_control_input(cart,pendulum,theta_dot,x_dot,g):
    # Using LQR to find control inputs
    
    # Implement the 4 x 4 system matrix A
    ### START CODE HERE ### ~ 4-6 lines of code
    
    ### END CODE HERE ###
    
    B = np.matrix(  [
                    [0],
                    [1/cart.mass],
                    [0],
                    [1/(pendulum.length*cart.mass)]
                    ])

    # The Q and R matrices are emperically tuned
    Q = np.matrix(  [
                    [10,0,0,0],
                    [0,1,0,0],
                    [0,0,10000,0],
                    [0,0,0,100]
                    ])

    R = np.matrix([500])

    # The K matrix is calculated using the lqr function 
    
    # Continuous Ricatti equation solution     
    K, S, E = lqr(A, B, Q, R)
     
    # Discrete Ricatti equation solution   
    #     K, S, E = dlqr(Ad, Bd, Q, R)

    # fill numerically the current state of the system     
    x = np.matrix(  [
                    [np.squeeze(np.asarray(cart.x))],
                    [np.squeeze(np.asarray(x_dot))],
                    [np.squeeze(np.asarray(pendulum.theta))],
                    [np.squeeze(np.asarray(theta_dot))]
                    ])
    
    # define a desired state for the system     
    desired = np.matrix([
                    [200],
                    [0],
                    [0],
                    [0]
                    ])
    
    # obtain forces according to the control law     
    F = -(K*(x-desired))
    return np.squeeze(np.asarray(F)), K

### Execute main function with applied force computed by the LQR controller

In [ ]:
%matplotlib notebook

main('lqr')

## References
https://en.wikipedia.org/wiki/PID_controller#Mathematical_form

https://danielpiedrahita.wordpress.com/portfolio/cart-pole-control/


Franklin, G.F., Powell, J.D., Emami-Naeini, A. and Powell, J.D., 1994. Feedback control of dynamic systems (Vol. 3). Reading, MA: Addison-Wesley.

code reference authors : Wouter Wolfslag, Nikhil Advani and Kostas Alexis

#### Authors : Jack Wilkinson and Theodoros Stouraitis